# ch5_データ理解からやり直し.ipynb

In [1]:
import warnings
warnings.filterwarnings('ignore')

import torch
import numpy as np
import pandas as pd
from pandas import DataFrame
from tqdm import tqdm
from sklearn.utils import check_random_state
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib
plt.style.use('ggplot')

from dataset import generate_synthetic_data
# from policylearners import RegBasedPolicyLearner, GradientBasedPolicyLearner, POTEC
from utils import softmax

In [15]:
from copy import copy
from collections import OrderedDict
from dataclasses import dataclass

import numpy as np
from sklearn.utils import check_random_state
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR

from utils import softmax, RegBasedPolicyDataset, GradientBasedPolicyDataset


@dataclass
class RegBasedPolicyLearner:
    """回帰ベースのアプローチに基づくオフ方策学習"""
    dim_x: int
    num_actions: int
    hidden_layer_size: tuple = (30, 30, 30)
    activation: str = "elu"
    batch_size: int = 16
    learning_rate_init: float = 0.005
    gamma: float = 0.98
    alpha: float = 1e-6
    log_eps: float = 1e-10
    solver: str = "adagrad"
    max_iter: int = 30
    random_state: int = 12345

    def __post_init__(self) -> None:
        """Initialize class."""
        layer_list = []
        input_size = self.dim_x

        if self.activation == "tanh":
            activation_layer = nn.Tanh
        elif self.activation == "relu":
            activation_layer = nn.ReLU
        elif self.activation == "elu":
            activation_layer = nn.ELU

        for i, h in enumerate(self.hidden_layer_size):
            layer_list.append(("l{}".format(i), nn.Linear(input_size, h)))
            layer_list.append(("a{}".format(i), activation_layer()))
            input_size = h
        layer_list.append(("output", nn.Linear(input_size, self.num_actions)))

        self.nn_model = nn.Sequential(OrderedDict(layer_list))

        self.random_ = check_random_state(self.random_state)
        self.train_loss = []
        self.train_value = []
        self.test_value = []

    def fit(self, dataset: dict, dataset_test: dict) -> None:
        x, a, r = dataset["x"], dataset["a"], dataset["r"]

        if self.solver == "adagrad":
            optimizer = optim.Adagrad(
                self.nn_model.parameters(),
                lr=self.learning_rate_init,
                weight_decay=self.alpha,
            )
        elif self.solver == "adam":
            optimizer = optim.AdamW(
                self.nn_model.parameters(),
                lr=self.learning_rate_init,
                weight_decay=self.alpha,
            )
        else:
            raise NotImplementedError("`solver` must be one of 'adam' or 'adagrad'")

        training_data_loader = self._create_train_data_for_opl(x, a, r)

        # start policy training
        scheduler = ExponentialLR(optimizer, gamma=self.gamma)
        q_x_a_train, q_x_a_test = dataset["q_x_a"], dataset_test["q_x_a"]
        for _ in range(self.max_iter):
            loss_epoch = 0.0
            self.nn_model.train()
            for x_, a_, r_ in training_data_loader:
                optimizer.zero_grad()
                q_hat = self.nn_model(x_)
                idx = torch.arange(a_.shape[0], dtype=torch.long)
                loss = ((r_ - q_hat[idx, a_]) ** 2).mean()
                loss.backward()
                optimizer.step()
                loss_epoch += loss.item()
            pi_train = self.predict(dataset)
            scheduler.step()
            self.train_value.append((q_x_a_train * pi_train).sum(1).mean())
            pi_test = self.predict(dataset_test)
            self.test_value.append((q_x_a_test * pi_test).sum(1).mean())
            self.train_loss.append(loss_epoch)

    def _create_train_data_for_opl(
        self,
        x: np.ndarray,
        a: np.ndarray,
        r: np.ndarray,
    ) -> tuple:
        dataset = RegBasedPolicyDataset(
            torch.from_numpy(x).float(),
            torch.from_numpy(a).long(),
            torch.from_numpy(r).float(),
        )

        data_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=self.batch_size,
        )

        return data_loader

    def predict(self, dataset_test: np.ndarray, beta: float = 10) -> np.ndarray:
        self.nn_model.eval()
        x = torch.from_numpy(dataset_test["x"]).float()
        q_hat = self.nn_model(x).detach().numpy()

        return softmax(beta * q_hat)

    def predict_q(self, dataset_test: np.ndarray) -> np.ndarray:
        self.nn_model.eval()
        x = torch.from_numpy(dataset_test["x"]).float()

        return self.nn_model(x).detach().numpy()


@dataclass
class GradientBasedPolicyLearner:
    """勾配ベースのアプローチに基づくオフ方策学習"""
    dim_x: int
    num_actions: int
    hidden_layer_size: tuple = (30, 30, 30)
    activation: str = "elu"
    batch_size: int = 16
    learning_rate_init: float = 0.005
    gamma: float = 0.98
    alpha: float = 1e-6
    imit_reg: float = 0.0
    log_eps: float = 1e-10
    solver: str = "adagrad"
    max_iter: int = 30
    random_state: int = 12345

    def __post_init__(self) -> None:
        """Initialize class."""
        layer_list = []
        input_size = self.dim_x

        if self.activation == "tanh":
            activation_layer = nn.Tanh
        elif self.activation == "relu":
            activation_layer = nn.ReLU
        elif self.activation == "elu":
            activation_layer = nn.ELU

        for i, h in enumerate(self.hidden_layer_size):
            layer_list.append(("l{}".format(i), nn.Linear(input_size, h)))
            layer_list.append(("a{}".format(i), activation_layer()))
            input_size = h
        layer_list.append(("output", nn.Linear(input_size, self.num_actions)))
        layer_list.append(("softmax", nn.Softmax(dim=1)))

        self.nn_model = nn.Sequential(OrderedDict(layer_list))

        self.random_ = check_random_state(self.random_state)
        self.train_loss = []
        self.train_value = []
        self.test_value = []

    def fit(self, dataset: dict, dataset_test: dict, q_hat: np.ndarray = None) -> None:
        x, a, r = dataset["x"], dataset["a"], dataset["r"]
        pscore, pi_0 = dataset["pscore"], dataset["pi_0"]
        if q_hat is None:
            q_hat = np.zeros((r.shape[0], self.num_actions))

        if self.solver == "adagrad":
            optimizer = optim.Adagrad(
                self.nn_model.parameters(),
                lr=self.learning_rate_init,
                weight_decay=self.alpha,
            )
        elif self.solver == "adam":
            optimizer = optim.AdamW(
                self.nn_model.parameters(),
                lr=self.learning_rate_init,
                weight_decay=self.alpha,
            )
        else:
            raise NotImplementedError("`solver` must be one of 'adam' or 'adagrad'")

        training_data_loader = self._create_train_data_for_opl(
            x,
            a,
            r,
            pscore,
            q_hat,
            pi_0,
        )

        # start policy training
        scheduler = ExponentialLR(optimizer, gamma=self.gamma)
        q_x_a_train, q_x_a_test = dataset["q_x_a"], dataset_test["q_x_a"]
        _list = []
        for _ in range(self.max_iter):
            loss_epoch = 0.0
            self.nn_model.train()
            for x_, a_, r_, p, q_hat_, pi_0_ in training_data_loader:
                optimizer.zero_grad()
                pi = self.nn_model(x_)
                loss = -self._estimate_policy_gradient(
                    a=a_,
                    r=r_,
                    pscore=p,
                    q_hat=q_hat_,
                    pi_0=pi_0_,
                    pi=pi,
                ).mean()
                iw, _estimated_policy_grad_arr = self._estimate_policy_gradient_2(
                    a=a_,
                    r=r_,
                    pscore=p,
                    q_hat=q_hat_,
                    pi_0=pi_0_,
                    pi=pi,
                )
                _list.append(loss)
                loss.backward()
                optimizer.step()
                loss_epoch += loss.item()
            self.train_loss.append(loss_epoch)
            scheduler.step()
            pi_train = self.predict(dataset)
            self.train_value.append((q_x_a_train * pi_train).sum(1).mean())
            pi_test = self.predict(dataset_test)
            self.test_value.append((q_x_a_test * pi_test).sum(1).mean())

        return _list

    def _create_train_data_for_opl(
        self,
        x: np.ndarray,
        a: np.ndarray,
        r: np.ndarray,
        pscore: np.ndarray,
        q_hat: np.ndarray,
        pi_0: np.ndarray,
    ) -> tuple:
        dataset = GradientBasedPolicyDataset(
            torch.from_numpy(x).float(),
            torch.from_numpy(a).long(),
            torch.from_numpy(r).float(),
            torch.from_numpy(pscore).float(),
            torch.from_numpy(q_hat).float(),
            torch.from_numpy(pi_0).float(),
        )

        data_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=self.batch_size,
        )

        return data_loader

    def _estimate_policy_gradient(
        self,
        a: torch.Tensor,
        r: torch.Tensor,
        pscore: torch.Tensor,
        q_hat: torch.Tensor,
        pi: torch.Tensor,
        pi_0: torch.Tensor,
    ) -> torch.Tensor:
        current_pi = pi.detach()
        log_prob = torch.log(pi + self.log_eps)
        idx = torch.arange(a.shape[0], dtype=torch.long)

        q_hat_factual = q_hat[idx, a]
        iw = current_pi[idx, a] / pscore
        estimated_policy_grad_arr = iw * (r - q_hat_factual) * log_prob[idx, a]
        estimated_policy_grad_arr += torch.sum(q_hat * current_pi * log_prob, dim=1)

        # imitation regularization
        estimated_policy_grad_arr += self.imit_reg * log_prob[idx, a]

        return estimated_policy_grad_arr

    def _estimate_policy_gradient_2(
        self,
        a: torch.Tensor,
        r: torch.Tensor,
        pscore: torch.Tensor,
        q_hat: torch.Tensor,
        pi: torch.Tensor,
        pi_0: torch.Tensor,
    ) -> torch.Tensor:
        current_pi = pi.detach()
        log_prob = torch.log(pi + self.log_eps)
        idx = torch.arange(a.shape[0], dtype=torch.long)

        q_hat_factual = q_hat[idx, a]
        iw = current_pi[idx, a] / pscore
        estimated_policy_grad_arr = iw * (r - q_hat_factual) * log_prob[idx, a]
        _estimated_policy_grad_arr = estimated_policy_grad_arr
        estimated_policy_grad_arr += torch.sum(q_hat * current_pi * log_prob, dim=1)

        # imitation regularization
        estimated_policy_grad_arr += self.imit_reg * log_prob[idx, a]

        return iw, _estimated_policy_grad_arr
    
    def predict(self, dataset_test: np.ndarray) -> np.ndarray:

        self.nn_model.eval()
        x = torch.from_numpy(dataset_test["x"]).float()
        return self.nn_model(x).detach().numpy()


@dataclass
class POTEC:
    """回帰ベースと勾配ベースのアプローチを融合した2段階方策学習"""
    dim_x: int
    num_actions: int
    num_clusters: int = 1
    hidden_layer_size: tuple = (30, 30, 30)
    activation: str = "elu"
    batch_size: int = 16
    learning_rate_init: float = 0.005
    gamma: float = 0.98
    alpha: float = 1e-6
    log_eps: float = 1e-10
    solver: str = "adagrad"
    max_iter: int = 30
    random_state: int = 12345

    def __post_init__(self) -> None:
        """Initialize class."""
        layer_list = []
        input_size = self.dim_x

        if self.activation == "tanh":
            activation_layer = nn.Tanh
        elif self.activation == "relu":
            activation_layer = nn.ReLU
        elif self.activation == "elu":
            activation_layer = nn.ELU

        for i, h in enumerate(self.hidden_layer_size):
            layer_list.append(("l{}".format(i), nn.Linear(input_size, h)))
            layer_list.append(("a{}".format(i), activation_layer()))
            input_size = h
        layer_list.append(("output", nn.Linear(input_size, self.num_clusters)))
        layer_list.append(("softmax", nn.Softmax(dim=1)))

        self.nn_model = nn.Sequential(OrderedDict(layer_list))

        self.random_ = check_random_state(self.random_state)
        self.train_loss = []
        self.train_value = []
        self.test_value = []

    def fit(
        self,
        dataset: dict,
        dataset_test: dict,
        f_hat: np.ndarray = None,
        f_hat_test: np.ndarray = None,
    ) -> None:
        x, a, r = dataset["x"], dataset["a"], dataset["r"]
        pscore_c, phi_a = dataset["pscore_c"], torch.from_numpy(dataset["phi_a"])
        if f_hat is None:
            f_hat = np.zeros(dataset["h_x_a"].shape)
        if f_hat_test is None:
            f_hat_test = np.zeros(dataset_test["h_x_a"].shape)

        if self.solver == "adagrad":
            optimizer = optim.Adagrad(
                self.nn_model.parameters(),
                lr=self.learning_rate_init,
                weight_decay=self.alpha,
            )
        elif self.solver == "adam":
            optimizer = optim.AdamW(
                self.nn_model.parameters(),
                lr=self.learning_rate_init,
                weight_decay=self.alpha,
            )
        else:
            raise NotImplementedError("`solver` must be one of 'adam' or 'adagrad'")

        training_data_loader = self._create_train_data_for_opl(
            x,
            a,
            r,
            pscore_c,
            f_hat,
            dataset["pi_0_c"],
        )

        # start policy training
        q_x_a_train, q_x_a_test = dataset["q_x_a"], dataset_test["q_x_a"]
        for _ in range(self.max_iter):
            loss_epoch = 0.0
            self.nn_model.train()
            for (x, a, r, p_c, f_hat_, _) in training_data_loader:
                optimizer.zero_grad()
                pi = self.nn_model(x)
                loss = -self._estimate_policy_gradient(
                    x=x,
                    a=a,
                    phi_a=phi_a,
                    r=r,
                    pscore_c=p_c,
                    f_hat=f_hat_,
                    pi=pi,
                ).mean()
                loss.backward()
                optimizer.step()
                loss_epoch += loss.item()
            self.train_loss.append(loss_epoch)
            pi_train = self.predict(dataset, f_hat)
            self.train_value.append((q_x_a_train * pi_train).sum(1).mean())
            pi_test = self.predict(dataset_test, f_hat_test)
            self.test_value.append((q_x_a_test * pi_test).sum(1).mean())

    def _create_train_data_for_opl(
        self,
        x: np.ndarray,
        a: np.ndarray,
        r: np.ndarray,
        pscore_c: np.ndarray,
        f_hat: np.ndarray,
        pi_0_c: np.ndarray,
    ) -> tuple:
        dataset = GradientBasedPolicyDataset(
            torch.from_numpy(x).float(),
            torch.from_numpy(a).long(),
            torch.from_numpy(r).float(),
            torch.from_numpy(pscore_c).float(),
            torch.from_numpy(f_hat).float(),
            torch.from_numpy(pi_0_c).float(),
        )

        data_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=self.batch_size,
        )

        return data_loader

    def _estimate_policy_gradient(
        self,
        x: torch.Tensor,
        a: torch.Tensor,
        r: torch.Tensor,
        phi_a: torch.Tensor,
        pscore_c: torch.Tensor,
        f_hat: torch.Tensor,
        pi: torch.Tensor,
    ) -> torch.Tensor:
        current_pi = pi.detach()
        log_prob = torch.log(pi + self.log_eps)
        idx = torch.arange(a.shape[0], dtype=torch.long)

        f_hat_factual = f_hat[idx, a]
        iw = current_pi[idx, phi_a[a]] / pscore_c
        estimated_policy_grad_arr = iw * (r - f_hat_factual) * log_prob[idx, phi_a[a]]

        f_hat_c = torch.zeros((a.shape[0], self.num_clusters))
        for c in range(self.num_clusters):
            if (phi_a == c).sum() > 0:
                f_hat_c[:, c] = f_hat[:, phi_a == c].max(1)[0]
            else:
                f_hat_c[:, c] = 0.0
        estimated_policy_grad_arr += torch.sum(f_hat_c * current_pi * log_prob, dim=1)

        return estimated_policy_grad_arr

    def predict(self, dataset_test: dict, f_hat_test: np.ndarray) -> np.ndarray:
        self.nn_model.eval()
        x = torch.from_numpy(dataset_test["x"]).float()
        pi_c = self.nn_model(x).detach().numpy()
        phi_a = torch.from_numpy(dataset_test["phi_a"])

        n = x.shape[0]
        action_set = np.arange(f_hat_test.shape[1])
        overall_policy = np.zeros(f_hat_test.shape)
        for c in range(self.num_clusters):
            if (phi_a == c).sum() > 0:
                best_actions_given_clusters = action_set[phi_a == c][
                    f_hat_test[:, phi_a == c].argmax(1)
                ]
                overall_policy[np.arange(n), best_actions_given_clusters] = pi_c[:, c]

        return overall_policy


In [20]:
## シミュレーション設定
num_runs = 100 # シミュレーションの繰り返し回数
dim_x = 3 # 特徴量xの次元
num_actions = 3 # 行動数, |A|
num_clusters = 2 # 行動クラスタ数, |C|
lambda_ = 0.5 # クラスタ効果と残差効果の配合率
max_iter = 31 # エポック数
test_data_size = 50000 # テストデータのサイズ
random_state = 12345
torch.manual_seed(random_state)
random_ = check_random_state(random_state)
num_data_list = [100, 200, 500, 1000, 2000] # トレーニングデータのサイズ
num_data = num_data_list[4]

## 期待報酬関数を定義するためのパラメータを抽出
phi_a = random_.choice(num_clusters, size=num_actions)
theta_g = random_.normal(size=(dim_x, num_clusters))
M_g = random_.normal(size=(dim_x, num_clusters))
b_g = random_.normal(size=(1, num_clusters))
theta_h = random_.normal(size=(dim_x, num_actions))
M_h = random_.normal(size=(dim_x, num_actions))
b_h = random_.normal(size=(1, num_actions))

## 学習された方策の真の性能を近似するためのテストデータを生成
test_data = generate_synthetic_data(
    num_data=test_data_size, lambda_=lambda_,
    theta_g=theta_g, M_g=M_g, b_g=b_g, theta_h=theta_h, M_h=M_h, b_h=b_h, phi_a=phi_a,
    dim_context=dim_x, num_actions=num_actions, num_clusters=num_clusters, random_state = random_state
)
pi_0_value = (test_data["q_x_a"] * test_data["pi_0"]).sum(1).mean()

_ = 0
offline_logged_data = generate_synthetic_data(
    num_data=num_data, lambda_=lambda_,
    theta_g=theta_g, M_g=M_g, b_g=b_g, theta_h=theta_h, M_h=M_h, b_h=b_h, phi_a=phi_a,
    dim_context=dim_x, num_actions=num_actions, num_clusters=num_clusters,
    random_state = _
)
x_df = pd.DataFrame(offline_logged_data['x'], columns=['x_0', 'x_1', 'x_2'])
a_df = pd.DataFrame(offline_logged_data['a'], columns=['a'])
c_df = pd.DataFrame(offline_logged_data['c'], columns=['c'])
r_df = pd.DataFrame(offline_logged_data['r'], columns=['r'])
pi_0_df = pd.DataFrame(offline_logged_data['pi_0'], columns=['pi_0_0', 'pi_0_1', 'pi_0_2'])
pi_0_c_df = pd.DataFrame(offline_logged_data['pi_0_c'], columns=['pi_0_c_0', 'pi_0_c_1'])
pscore_df = pd.DataFrame(offline_logged_data['pscore'], columns=['pscore'])
pscore_c_df = pd.DataFrame(offline_logged_data['pscore_c'], columns=['pscore_c'])
g_x_c_df = pd.DataFrame(offline_logged_data['g_x_c'], columns=['g_x_c_0', 'g_x_c_1'])
h_x_a_df = pd.DataFrame(offline_logged_data['h_x_a'], columns=['h_x_a_0', 'h_x_a_1', 'h_x_a_2'])
q_x_a_df = pd.DataFrame(offline_logged_data['q_x_a'], columns=['q_x_a_0', 'q_x_a_1', 'q_x_a_2'])

offline_logged_data_df = pd.concat([
    x_df, a_df, c_df, r_df, pi_0_df, pi_0_c_df, pscore_df, pscore_c_df, g_x_c_df, h_x_a_df, q_x_a_df
], axis=1)
offline_logged_data_df

,x_0,x_1,x_2,a,c,r,pi_0_0,pi_0_1,pi_0_2,pi_0_c_0,...,pscore,pscore_c,g_x_c_0,g_x_c_1,h_x_a_0,h_x_a_1,h_x_a_2,q_x_a_0,q_x_a_1,q_x_a_2
0,1.764052,0.400157,0.978738,1,1,0,0.330557,0.438512,0.230930,0.330557,...,0.438512,0.669443,0.499947,0.012539,0.011047,0.380021,0.095861,0.510994,0.392559,0.108399
1,2.240893,1.867558,-0.977278,2,1,1,0.154751,0.483056,0.362193,0.154751,...,0.362193,0.845249,0.500000,0.000067,0.244939,0.499999,0.496463,0.744939,0.500066,0.496530
2,0.950088,-0.151357,-0.103219,2,1,0,0.292559,0.309976,0.397465,0.292559,...,0.397465,0.707441,0.377560,0.277070,0.378410,0.405722,0.415666,0.755970,0.682791,0.692736
3,0.410599,0.144044,1.454274,1,1,1,0.491739,0.376881,0.131380,0.491739,...,0.376881,0.508261,0.492283,0.396090,0.499219,0.302622,0.498354,0.991502,0.698712,0.894444
4,0.761038,0.121675,0.443863,1,1,1,0.254738,0.260212,0.485050,0.254738,...,0.260212,0.745262,0.228768,0.356311,0.328859,0.289922,0.411425,0.557627,0.646232,0.767735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-0.484180,0.207029,0.754987,2,1,1,0.319816,0.339420,0.340763,0.319816,...,0.340763,0.680184,0.323749,0.208238,0.295656,0.174609,0.391237,0.619405,0.382847,0.599474
1996,1.359420,0.696072,0.682201,1,1,0,0.438499,0.322525,0.238976,0.438499,...,0.322525,0.561501,0.487573,0.140192,0.109516,0.335420,0.316686,0.597089,0.475613,0.456878
1997,-0.011214,1.344760,-0.831492,1,1,1,0.410658,0.359443,0.229899,0.410658,...,0.359443,0.589342,0.485086,0.270239,0.499298,0.498875,0.499573,0.984384,0.769114,0.769812
1998,-0.407972,-1.330804,0.352599,1,1,0,0.468760,0.269751,0.261489,0.468760,...,0.269751,0.531240,0.347049,0.065895,0.416049,0.399401,0.037246,0.763098,0.465296,0.103141


In [21]:
# 報酬実績値
result_r = offline_logged_data_df['r'].mean()
print(f'報酬実績値: {result_r:.3f}')

報酬実績値: 0.635


In [22]:
### 回帰ベースのアプローチ
reg = RegBasedPolicyLearner(dim_x=dim_x, num_actions=num_actions, max_iter=max_iter)
reg.fit(offline_logged_data, test_data)
pi_reg = reg.predict(test_data)
true_value_of_learned_policies_reg = (test_data["q_x_a"] * pi_reg).sum(1).mean()
print(f'回帰ベースのアプローチによる報酬値: {true_value_of_learned_policies_reg:.3f}')

回帰ベースのアプローチによる報酬値: 0.746


In [23]:
offline_logged_data['phi_a']

array([0, 1, 1])